In [1]:
#导入所需库
import requests as req
from bs4 import BeautifulSoup
import re
import  os

In [2]:
def save_data(title_list=[],content_list=[]):
    '''
    将爬取到的内容存到当前目录的blog目录下，如果没有blog
    目录则创建
    :title_list: 文章题目列表
    :content_list: 文章内容列表
    '''
    path=r'./blog/'
    if not os.path.exists(path):
        os.mkdir(path)
    for i in range(len(title_list)):
        with open(path+title_list[i]+'.txt','w+',encoding='utf-8') as f:
            f.write(str(content_list[i]))
   
    

In [3]:
def spider_run(blog_name='qq_42304949',num_page=3):
    '''
    爬取博客文章链接及内容并存储，成功返回1，反之返回0
    :blog_name: 博客ID
    :numpage: 爬取页数
    '''
    headers = {
            'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) \
            Chrome/75.0.3770.80 Safari/537.36'
    }
    
    for i in range(num_page):
        url='https://blog.csdn.net/' + blog_name + '/article/list/'+str(i+1)+'?' 
        try:
            res=req.get(url,timeout=10)
            res.raise_for_status()
            res.encoding=res.apparent_encoding
            #print(len(res.text),i)
            #print(res.text)

        except Exception as e:
            print('Exception:',e)

        soup=BeautifulSoup(res.text,'lxml')
        reg=re.compile(r'原创</span>\s*(.*)      </a>')    #正则表达式匹配文章标题       
        title_list=reg.findall(res.text)
        #print(titlelist)
        content_list=soup.select('p a')          #BeautifulSoup的css选择器选择文章链接及文章开头部分内容
        #content_list=soup.find_all('p',attrs='content') 
        #print(soup.select('p a'))
        save_data(title_list,content_list)       #保存
        if len(title_list)==0:
            return 0
    return 1



In [4]:
#GUI部分
from tkinter import *

def run():
    s1='爬取成功,已存入目录：./blog/\n'
    s2='爬取失败！请检查网络是否畅通或输入是否合理\n'
    s=s2
    if(spider_run(inp1.get(),num_page=int(inp2.get()))):  s=s1
    txt.insert(END, s)    # 追加显示运算结果
    #inp1.delete(0, END)   # 清空输入


root = Tk()
root.geometry('360x250')
root.title('blog_csdn_tool')

lb1 = Label(root, text=' 请输入CSDN博客ID')          #文本标签1
lb1.place(relx=0.20, rely=0.1, relwidth=0.8, relheight=0.1)   
default_value1 = StringVar();  default_value1.set('qq_42304949');        #设置默认值
inp1 = Entry(root,textvariable=default_value1)                            #输入文本框1
inp1.place(relx=0.1, rely=0.1, relwidth=0.3, relheight=0.12)   

lb2 = Label(root, text=' 请输入爬取页数')          #文本标签2
lb2.place(relx=0.15, rely=0.3, relwidth=0.9, relheight=0.1)
default_value2 = StringVar();  default_value2.set('3');        #设置默认值
inp2 = Entry(root,textvariable=default_value2)                    #输入文本框2
inp2.place(relx=0.1, rely=0.3, relwidth=0.3, relheight=0.12)   

# 直接调用 run()
btn1 = Button(root, text='开始爬取', command=run)             #"开始爬取按钮   
btn1.place(relx=0.35, rely=0.45, relwidth=0.3, relheight=0.1)


# 在窗体垂直自上而下位置60%处起，布局相对窗体高度40%高的文本框    
txt = Text(root)                                      #输出文本框
txt.place(rely=0.6, relheight=0.3)
root.mainloop()                 #死循环，直到关闭窗口
